In [1]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.ml.feature import IndexToString, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
# 设置运行环境

conf = SparkConf()
conf.setAppName("CF")
conf.setMaster("spark://172.19.99.121:7077")
#sc.stop()
sc = SparkContext(conf = conf)
sqlCtx = SQLContext(sc)

In [4]:
# 从数据集读取数据作为处理数据源，在此基础上创建 DataFrame

data_path = "dataset.csv"
datasetRDD = sc.textFile(data_path).map(lambda line:line.split("::"))
datasetRDD = datasetRDD.map(lambda line:(int(line[0]), line[1], line[2], float(line[3])))

dataDF = sqlCtx.createDataFrame(datasetRDD)
dataDF = dataDF.toDF("user_id", "user_name", "problem_name", "ac_rate")

In [5]:
datasetRDD.take(10)

[(1, 'tourist', 'C - Party', 4.2105),
 (1, 'tourist', 'F - Koala and Notebook', 2.0),
 (1, 'tourist', 'G1 - Into Blocks (easy version)', 2.0),
 (1, 'tourist', 'E2 - Rotate Columns (hard version)', 2.0),
 (1, 'tourist', 'E1 - Rotate Columns (easy version)', 2.0),
 (1, 'tourist', 'D - Cow and Snacks', 2.0),
 (1, 'tourist', 'C - Paint the Digits', 5.0),
 (1, 'tourist', 'B - Koala and Lights', 2.0),
 (1, 'tourist', 'A - Paint the Numbers', 2.0),
 (1, 'tourist', 'I - Unusual Graph', 2.0)]

In [6]:
# 将原始的 {problem_name} 转换成数值类型

problemIndexer = StringIndexer(inputCol="problem_name",outputCol="problem_id").fit(dataDF)


In [7]:
# 使用 Pipeline 对数据进行处理  为数据添加 problem_id

pipeline = Pipeline(stages=[problemIndexer])
datasetDF = pipeline.fit(dataDF).transform(dataDF)

In [8]:
# 将数据按照比例随机划分为训练集和测试集

import time

training_data,test_data = datasetDF.randomSplit([0.8, 0.2], seed=time.time())

In [9]:
# 使用 ASL 训练推荐系统模型

# 显性反馈
alsExplicit = ALS(rank=10,
                  maxIter=10,
                  regParam=0.20,
                  userCol="user_id", 
                  itemCol="problem_id",
                  ratingCol="ac_rate",
                  implicitPrefs=False,
                  nonnegative=False,
                  coldStartStrategy="drop") # 设置 冷启动 确保没有NAN的值

modelExplicit = alsExplicit.fit(training_data)


# 隐性反馈
alsImplicit = ALS(rank=10,
                  maxIter=10,
                  regParam=0,
                  userCol="user_id",
                  itemCol="problem_id",
                  ratingCol="ac_rate",
                  implicitPrefs=True,
                  nonnegative=False,
                  coldStartStrategy="drop")  # 设置 冷启动 确保没有NAN的值

modelImplicit = alsImplicit.fit(training_data)


In [10]:
# 测试数据集上的测试模型 给予展示

# 显性反馈
predictionExplicit = modelExplicit.transform(test_data)
predictionExplicit.select("user_name", "problem_name", "ac_rate", "prediction").show()


+------------------+--------------------+-------+----------+
|         user_name|        problem_name|ac_rate|prediction|
+------------------+--------------------+-------+----------+
|           nvmdava|A - Game With Sticks|    2.0| 2.4346688|
|    abhay7bhardwaj|A - Game With Sticks|    5.0| 2.3345034|
|             sahal|A - Game With Sticks|    5.0| 2.5367494|
|               mi_|A - Game With Sticks|    2.0| 2.5611837|
|           1807006|A - Game With Sticks|    2.0| 2.8562415|
|       Toirov_Sadi|A - Game With Sticks| 3.3333| 2.7396178|
|      kamesh.joshi|A - Game With Sticks|    2.0| 2.7287405|
|         Nishabila|A - Game With Sticks| 3.3333| 2.6567192|
|            Monzur|A - Game With Sticks|    4.0| 2.6132348|
|giorgi.adeishvili2|A - Game With Sticks|   1.25| 2.9471872|
|         piyal_043|A - Game With Sticks|    2.0|  2.674006|
|    innocent_coder|A - Game With Sticks|    2.0| 2.8207867|
|            gmdsam|A - Game With Sticks|    2.0| 2.5837677|
|       DebSourav33|A - 

In [10]:
# 隐性反馈

predictionImplicit = modelImplicit.transform(test_data)
predictionImplicit.select("user_name", "problem_name", "ac_rate", "prediction").show()

+--------------------+--------------------+-------+----------+
|           user_name|        problem_name|ac_rate|prediction|
+--------------------+--------------------+-------+----------+
|            icecuber|C - Permutation r...|    2.0| 1.0823808|
|StillBlueSkyOverhead|C - Permutation r...|    5.0|0.54611164|
|            beginend|C - Permutation r...| 1.6667| 0.7306187|
|          yokozuna57|C - Permutation r...|    5.0|0.33128878|
|             zscoder|C - Permutation r...|    2.0| 0.6208643|
|         UtahaSenpai|C - Permutation r...| 3.3333|0.50730187|
|                Arpa|C - Permutation r...|    2.0| 0.6767923|
|              upobir|C - Permutation r...|    5.0| 0.5550964|
|          egor.lifar|C - Permutation r...| 1.1111|0.72142345|
|            wucstdio|C - Permutation r...|    4.0|0.85658467|
|        AlexDmitriev|C - Permutation r...|    5.0|0.56264424|
|               Torta|C - Permutation r...| 3.3333|0.58635855|
|          zhouzhidao|C - Permutation r...|    4.0| 0.4

In [11]:
# 使用 RegressionEvaluator 进行模型评估

evalutor = RegressionEvaluator(predictionCol='prediction',
                              labelCol='ac_rate',
                              metricName='rmse')

# 显性模型评估
rmseExplicit = evalutor.evaluate(predictionExplicit)
print("Explict: Root-mean-square error = " + str(rmseExplicit))

# 隐性模型评估
rmseImplicit = evalutor.evaluate(predictionImplicit)
print("Implict: Root-meam-square error = " + str(rmseImplicit))

Explict: Root-mean-square error = 1.223525971425664
Implict: Root-meam-square error = 2.585368605445152


In [12]:
# 使用显性反馈模拟 对于所有的用户推荐题目
user_recs = modelExplicit.recommendForAllUsers(3)

In [13]:
items = user_recs.select("user_id", "recommendations.problem_id", "recommendations.rating")

In [14]:
items.show(10, False)

+-------+---------------------+---------------------------------+
|user_id|problem_id           |rating                           |
+-------+---------------------+---------------------------------+
|1580   |[13221, 13723, 13564]|[4.909134, 4.909134, 4.8953013]  |
|5300   |[13564, 14244, 13221]|[4.974255, 4.9506755, 4.9434695] |
|21220  |[13564, 14153, 13768]|[4.962142, 4.9615493, 4.958614]  |
|21700  |[13221, 13723, 13564]|[3.6338565, 3.6338565, 3.6279285]|
|30970  |[13768, 14153, 13564]|[4.988101, 4.9733377, 4.94595]   |
|35820  |[13564, 14244, 13221]|[4.9149885, 4.8936396, 4.8793297]|
|41890  |[13768, 14153, 13753]|[5.0938563, 5.0400815, 4.974268] |
|54190  |[13564, 14244, 13221]|[5.497941, 5.4805136, 5.467965]  |
|471    |[13564, 13723, 13221]|[4.817204, 4.8141236, 4.8141236] |
|1591   |[13221, 13723, 13564]|[4.750735, 4.750735, 4.7452836]  |
+-------+---------------------+---------------------------------+
only showing top 10 rows



In [15]:
sc.stop()